In [1709]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


In [1710]:
from cloudant.client import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey
from datetime import datetime
from faker import Faker
import random
import json

In [1711]:
username =  "b47f2543-95d8-4c01-b864-cf9fcb230ba7-bluemix"
password = "N6i0LnK7GlPfKyf1w2vo_pWYGP254k-g6Y78d32LiDx6"
client = Cloudant.iam(username, password)
client.connect()
databaseName = "maindatabase"
mainDatabase = client.create_database(databaseName)

In [1712]:
ListOfGooglePlaceID = []
ListOfGooglePlaceID.append("ChIJs--MqP1YwokRBwAhjXWIHn8")
ListOfGooglePlaceID.append("ChIJOwg_06VPwokRYv534QaPC8g")
ListOfGooglePlaceID.append("ChIJ9TRDoBBawokR5gdcflPqvjE")
ListOfGooglePlaceID.append("ChIJlYpIkBBawokRjyd8kYQpUrc")
ListOfGooglePlaceID.append("ChIJUXPOvRBawokR2iT8sI7Mq3U")
ListOfGooglePlaceID.append("ChIJh-L3uxlawokRDwLmkKwLIvY")
ListOfGooglePlaceID.append("ChIJPTJohhdawokR5-M2cwZ2WDo")
ListOfGooglePlaceID.append("ChIJ-8HrdxtawokREfw5AkGs_Os")
ListOfGooglePlaceID.append("ChIJ-ziaVBZawokRj7B7C4qAWPY")
ListOfGooglePlaceID.append("ChIJNfKP5xdawokRyCFSWap_ogE")
ListOfGooglePlaceID.append("ChIJ_7B6nRZawokRRg3UBR5XohI")
ListOfGooglePlaceID.append("ChIJ0fci9hNawokRJVR9hdTAt80")
ListOfGooglePlaceID.append("ChIJ9TRDoBBawokRJgmlJa_UJ0E")
ListOfGooglePlaceID.append("ChIJbd_WExZawokRbBhzy1Hc4Ms")
ListOfGooglePlaceID.append("ChIJuydjIhZawokRG1-vnBMShEw")
ListOfGooglePlaceID.append("ChIJ7UBKbh9awokRsLNXAfxLoHs")
ListOfGooglePlaceID.append("ChIJ7xcL8z1awokRPFmLkB9tXHc")
ListOfGooglePlaceID.append("ChIJ3blcJRRawokRex4mbeb9Ix4")
ListOfGooglePlaceID.append("ChIJl4BZ5opZwokRUR0n7MrTIEA")
ListOfGooglePlaceID.append("ChIJodiAGRZawokRHXCBqdWXV54")
ListOfGooglePlaceID.append("ChIJ3a-_JdJQwokR2SXNohPwSQI")





In [1713]:
geoJsonNyc1 = {"coordinates": [-74.006058, 40.712772], "type":"Point"}
geoJsonNyc2 = {"coordinates": [-74.005973, 40.712775], "type":"Point"}
geoJsonNyc3 = {"coordinates": [-74.014671, 40.709504], "type":"Point"}
geoJsonNyc4 = {"coordinates": [-74.013651, 40.709137], "type":"Point"}
geoJsonNyc5 = {"coordinates": [-74.013651, 40.709137], "type":"Point"}
geoJsonNyc6 = {"coordinates": [-74.010333, 40.711114], "type":"Point"}
geoJsonNyc7 = {"coordinates": [-74.009123, 40.715036], "type":"Point"}
geoJsonNyc8 = {"coordinates": [-74.015840, 40.706762], "type":"Point"}
geoJsonNyc9 = {"coordinates": [-74.008945, 40.706762], "type":"Point"}
geoJsonNyc10 = {"coordinates": [-74.007214, 40.707920], "type":"Point"}
geoJsonNyc11 = {"coordinates": [-74.010322, 40.704642], "type":"Point"}
geoJsonNyc12 = {"coordinates": [-74.012354, 40.703950], "type":"Point"}
geoJsonNyc13 = {"coordinates": [-74.013888, 40.709797], "type":"Point"}
geoJsonNyc14 = {"coordinates": [-74.007936, 40.705129], "type":"Point"}
geoJsonNyc15 = {"coordinates": [-74.007189, 40.705562], "type":"Point"}
geoJsonNyc16 = {"coordinates": [-74.007367, 40.715947], "type":"Point"}
geoJsonNyc17 = {"coordinates": [-74.006177, 40.705063], "type":"Point"}
geoJsonNyc18 = {"coordinates": [-74.010336, 40.704129], "type":"Point"}
geoJsonNyc19 = {"coordinates": [-74.012052, 40.715534], "type":"Point"}
geoJsonNyc20 = {"coordinates": [-74.011265, 40.706877], "type":"Point"}
geoJsonNyc21 = {"coordinates": [-74.043143, 40.717755], "type":"Point"}


geoJsonList = []
geoJsonList.append(geoJsonNyc1)
geoJsonList.append(geoJsonNyc2)
geoJsonList.append(geoJsonNyc3)
geoJsonList.append(geoJsonNyc4)
geoJsonList.append(geoJsonNyc5)
geoJsonList.append(geoJsonNyc6)
geoJsonList.append(geoJsonNyc7)
geoJsonList.append(geoJsonNyc8)
geoJsonList.append(geoJsonNyc9)
geoJsonList.append(geoJsonNyc10)
geoJsonList.append(geoJsonNyc11)
geoJsonList.append(geoJsonNyc12)
geoJsonList.append(geoJsonNyc13)
geoJsonList.append(geoJsonNyc14)
geoJsonList.append(geoJsonNyc15)
geoJsonList.append(geoJsonNyc16)
geoJsonList.append(geoJsonNyc17)
geoJsonList.append(geoJsonNyc18)
geoJsonList.append(geoJsonNyc19)
geoJsonList.append(geoJsonNyc20)
geoJsonList.append(geoJsonNyc21)
print(geoJsonList)





[{'coordinates': [-74.006058, 40.712772], 'type': 'Point'}, {'coordinates': [-74.005973, 40.712775], 'type': 'Point'}, {'coordinates': [-74.014671, 40.709504], 'type': 'Point'}, {'coordinates': [-74.013651, 40.709137], 'type': 'Point'}, {'coordinates': [-74.013651, 40.709137], 'type': 'Point'}, {'coordinates': [-74.010333, 40.711114], 'type': 'Point'}, {'coordinates': [-74.009123, 40.715036], 'type': 'Point'}, {'coordinates': [-74.01584, 40.706762], 'type': 'Point'}, {'coordinates': [-74.008945, 40.706762], 'type': 'Point'}, {'coordinates': [-74.007214, 40.70792], 'type': 'Point'}, {'coordinates': [-74.010322, 40.704642], 'type': 'Point'}, {'coordinates': [-74.012354, 40.70395], 'type': 'Point'}, {'coordinates': [-74.013888, 40.709797], 'type': 'Point'}, {'coordinates': [-74.007936, 40.705129], 'type': 'Point'}, {'coordinates': [-74.007189, 40.705562], 'type': 'Point'}, {'coordinates': [-74.007367, 40.715947], 'type': 'Point'}, {'coordinates': [-74.006177, 40.705063], 'type': 'Point'},

In [1714]:
def randomizeCondition() :
    num = random.randint(0, 2)
    if (num == 0) :
        return "Recovered/NoCovid19"
    if (num == 1) :
        return "PotentialCovid19"
    if (num == 2) :
        return "HasCovid19"
status = randomizeCondition()

def randomizeVisitID() :
    
    return random.randint(0, 999999999)

visit_id = randomizeVisitID()
print(visit_id)

def initListVisitDictionary(geoJsonList) :
    visitListDictionary = []
    for i in range(21):
        visitListDictionary.append({str(geoJsonList[i]["coordinates"]):[randomizeVisitID(), datetime.__str__(datetime(2020, 3, 13)), randomizeCondition()]})     
    return visitListDictionary

ListVisitDictionary = initListVisitDictionary(geoJsonList)


def randomizeTransactionID() :
    return random.randint(0, 32800000)

def randomizeContributionID() :
    return random.randint(0, 32800000)

def init_ListJsonDocument(ListVisitDictionary, geoJsonList):
    ListJsonDocument = []
    for i in range(len(geoJsonList)):
        jsonDocument = {
            "TransactionID" : randomizeTransactionID(),
            "ContributorID" : randomizeContributionID(),
            "GooglePlaceID": ListOfGooglePlaceID[i],
            "geometry": geoJsonList[i],
            "VisitDictionary": ListVisitDictionary[i]     
        }
        type(jsonDocument)
        ListJsonDocument.append(jsonDocument)
    return ListJsonDocument



332862120


In [1715]:
print(ListVisitDictionary)

[{'[-74.006058, 40.712772]': [829546505, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'[-74.005973, 40.712775]': [683392916, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'[-74.014671, 40.709504]': [580012845, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'[-74.013651, 40.709137]': [328233462, '2020-03-13 00:00:00', 'HasCovid19']}, {'[-74.013651, 40.709137]': [274103840, '2020-03-13 00:00:00', 'HasCovid19']}, {'[-74.010333, 40.711114]': [940573736, '2020-03-13 00:00:00', 'HasCovid19']}, {'[-74.009123, 40.715036]': [914298822, '2020-03-13 00:00:00', 'HasCovid19']}, {'[-74.01584, 40.706762]': [237530908, '2020-03-13 00:00:00', 'Recovered/NoCovid19']}, {'[-74.008945, 40.706762]': [945002601, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'[-74.007214, 40.70792]': [267671489, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'[-74.010322, 40.704642]': [900001056, '2020-03-13 00:00:00', 'PotentialCovid19']}, {'[-74.012354, 40.70395]': [48118897, '2020-03-13 00:00:00', 'HasCovid19']}, {'[-74.

In [1716]:
def initListIBMDataBaseDocument(ListJsonDocument) :
    ListIBMDataBaseDocument = []
    for i in ListJsonDocument:
        newDocument = mainDatabase.create_document(i)
        ListIBMDataBaseDocument.append(newDocument)
    return ListIBMDataBaseDocument
ListIBMDataBaseDocument = initListIBMDataBaseDocument(ListJsonDocument)

In [1717]:
print(ListIBMDataBaseDocument)

[{'TransactionID': 22408149, 'ContributorID': 24801727, 'GooglePlaceID': 'ChIJs--MqP1YwokRBwAhjXWIHn8', 'geometry': {'coordinates': [40.712772, -74.006058], 'type': 'Point'}, 'VisitDictionary': {'[40.712772, -74.006058]': [508311112, '2020-03-13 00:00:00', 'HasCovid19']}, '_id': '3bb25ccc8742d540f2b0bcf2dd99c8c8', '_rev': '1-4e920a6c264f845a3cb117151394237c'}, {'TransactionID': 4874235, 'ContributorID': 30647152, 'GooglePlaceID': 'ChIJOwg_06VPwokRYv534QaPC8g', 'geometry': {'coordinates': [40.712775, -74.005973], 'type': 'Point'}, 'VisitDictionary': {'[40.712775, -74.005973]': [251673167, '2020-03-13 00:00:00', 'PotentialCovid19']}, '_id': '710e24f85b3431577c382d15d38732ee', '_rev': '1-71067e9b965e38718068e3440a380525'}, {'TransactionID': 32276609, 'ContributorID': 17660365, 'GooglePlaceID': 'ChIJ9TRDoBBawokR5gdcflPqvjE', 'geometry': {'coordinates': [40.709504, -74.014671], 'type': 'Point'}, 'VisitDictionary': {'[40.709504, -74.014671]': [529390308, '2020-03-13 00:00:00', 'Recovered/NoC

In [1718]:
def severityOfArea(ListJsonDocument) :
    locationSeverityDict = {}
    highCount = 0
    mediumCount = 0
    lowCount = 0
    highWeight = 1
    mediumWeight = 0.5
    weightedSum = 0
    lowWeight = 0
    for jsonDocument in (ListJsonDocument) :
        weightedSum = 0
        keyJson = jsonDocument["geometry"]["coordinates"]
        print(keyJson)
        if (jsonDocument["VisitDictionary"][str(keyJson)][2].lower() == "HasCovid19".lower()):
            highCount = highCount + 1
        elif (jsonDocument["VisitDictionary"][str(keyJson)][2].lower() == "PotentialCovid19".lower()):
            mediumCount = mediumCount + 1
        else:
            lowCount = lowCount + 1
        weightedSum = weightedSum + ((highCount * highWeight) + (mediumCount * mediumWeight) + (lowCount * lowWeight))
        if tuple(keyJson) in locationSeverityDict:
            previousWeight = locationSeverityDict[tuple(keyJson)]
            locationSeverityDict[tuple(keyJson)] = previousWeight + weightedSum
        else:
            locationSeverityDict[tuple(keyJson)] = weightedSum
            
                
    return locationSeverityDict
        
        
        

LocationSeverityDict = severityOfArea(ListJsonDocument)
print(LocationSeverityDict)

[40.712772, -74.006058]
[40.712775, -74.005973]
[40.709504, -74.014671]
[40.709137, -74.013651]
[40.709137, -74.013651]
[40.711114, -74.010333]
[40.708977, -74.009123]
[40.715036, -74.01584]
[40.706762, -74.008945]
[40.70792, -74.007214]
[40.704642, -74.010322]
[40.70395, -74.012354]
[40.709797, -74.013888]
[40.705129, -74.007936]
[40.705562, -74.007189]
[40.715947, -74.007367]
[40.705063, -74.006177]
[40.704129, -74.010336]
[40.715534, -74.012052]
[40.706877, -74.011265]
[40.717755, -74.043143]
{(40.712772, -74.006058): 1.0, (40.712775, -74.005973): 1.5, (40.709504, -74.014671): 1.5, (40.709137, -74.013651): 3.0, (40.711114, -74.010333): 2.5, (40.708977, -74.009123): 3.0, (40.715036, -74.01584): 4.0, (40.706762, -74.008945): 5.0, (40.70792, -74.007214): 5.5, (40.704642, -74.010322): 6.0, (40.70395, -74.012354): 6.5, (40.709797, -74.013888): 6.5, (40.705129, -74.007936): 6.5, (40.705562, -74.007189): 6.5, (40.715947, -74.007367): 6.5, (40.705063, -74.006177): 7.5, (40.704129, -74.01033